#Суть дослідження:

**Мета:** дослідити діяльність компанії в розрізі географії,сформувати поради для подальшого розвитку, знайти нові можливості для розвитку та відобразити всі результати у вигляді зручного інтуітивно зрозумілого дашборда.Будемо досліджувати три географічні розріза: континенти, суб-регіони, країни.  
**Задачи:**  
1.Дослідити загальні метрики компанії в усіх географічних розрізах,виявити патерни взаємозв'язків.  
2.Дослідити прибуток по браузерам, операційним системам та девайсам в усіх географічних розрізах, за наявності таких знайти цікаві з точки зору бізнеса аномалії.  
3.Дослідити ефективність каналів трафіку та сформувати рекомендації по їх подальшому розвитку в усіх географічних розрізах.  
4.Знайти найприбутковіші категорії та дослідити чи зберігається ця тенденція на всіх географічних розрізах.  
5.Провести дослідження за індексом афінності , щоб зрозуміти, на яких ринках робити акцент на тій чи іншій категорії.  
6.Пошукати взаємозв'язкі та провести оцінку їх статистичної значущості як для глобальних метрик, так і поглиблено для пошуку більш  скритих інсайтів.  
7.Cформувати загальні висновки.  
8.Побудувати інтуітивно зрозумілий дашборд в Tablaue  
**План виконання:**  
1.Зробити всі необхідні підключення.  
2.Сформувати датасет.  
3.Очистити дані та провести EDA аналіз.  
4.Проаналізувати за допомогою графіків загальні метрики компанії в усіх географічних розрізах.  
5.Проаналізувати за допомогою графіків всі необхідні категоріальні дані в усіх розрізах географії(подвійна агрегація).  
6.Провести пошук,перевірку та надійність взаємозв'язків.  
7.Сформувати загальні висновки та рекомендації на основі бізнес логіки.



#Технічні підключення:

In [1]:
!pip install --upgrade google-cloud-bigquery

In [2]:
!pip install --upgrade plotly>=5.0.0 ipywidgets>=7.0.0

In [3]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import files,output
output.enable_custom_widget_manager()

import pandas as pd
import numpy as np

import scipy.stats as st

import kagglehub
import os

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.renderers.default = "colab"
from IPython.display import clear_output, display,IFrame

import ipywidgets as widgets

path = kagglehub.dataset_download("andradaolteanu/country-mapping-iso-continent-region") # цей датасет нам знадобиться щоб проаналізувати інформацію в розрізі суб-регіонів

Using Colab cache for faster access to the 'country-mapping-iso-continent-region' dataset.


In [4]:
df_countries = pd.read_csv('/kaggle/input/country-mapping-iso-continent-region/continents2.csv')
df_countries.rename(columns = {'name': 'country'}, inplace = True) # перейменуємо щоб мати спільне поле для join'a
df_countries.head()

,country,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [5]:
auth.authenticate_user()

#SQl Запит:

In [6]:
client = bigquery.Client(project="data-analytics-mate")

query = """
WITH product_order AS (
       SELECT *
       FROM data-analytics-mate.DA.order o
       LEFT JOIN data-analytics-mate.DA.product pr
       USING(item_id)
--всі session_id унікальні--
),
acount_info AS (
       SELECT accs.account_id,
              ga_session_id,
              is_verified,
              is_unsubscribed
       FROM data-analytics-mate.DA.account_session accs
       JOIN data-analytics-mate.DA.account ac
       ON accs.account_id = ac.id
),
session_date AS (
       SELECT *
       FROM data-analytics-mate.DA.session_params sp
       LEFT JOIN data-analytics-mate.DA.session s
       USING(ga_session_id)
),
combine_data AS (
       SELECT *
       FROM session_date sd
       LEFT JOIN product_order po
       USING(ga_session_id)
       LEFT JOIN acount_info aif
       USING(ga_session_id)
)
SELECT *
FROM combine_data
"""

query_job =client.query(query)
results = query_job.result()


In [7]:
df = results.to_dataframe()
df.head()

,ga_session_id,device,mobile_model_name,operating_system,language,browser,continent,country,medium,name,channel,date,item_id,name_1,category,price,short_description,account_id,is_verified,is_unsubscribed
0,7268004754,desktop,Chrome,Web,en-gb,Chrome,(not set),(not set),(none),(direct),Direct,2021-01-19,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>
1,8321293308,desktop,Chrome,Web,None,Chrome,(not set),(not set),(none),(direct),Direct,2021-01-06,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>
2,4392362373,desktop,Chrome,Web,None,Chrome,(not set),(not set),organic,(organic),Organic Search,2020-11-05,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>
3,140108698,desktop,Safari,Web,en-us,Chrome,(not set),(not set),(none),(direct),Direct,2020-12-09,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>
4,8351127264,mobile,iPhone,Web,en-us,Safari,(not set),(not set),organic,(organic),Organic Search,2020-11-10,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>


Поєднуємо датасети, щоб мати інформацію про суб-регіон:

In [8]:
df = pd.merge(df, df_countries[['country','sub-region']], on = 'country', how = 'left')

In [9]:
df.to_csv("my_data.csv", index=False, encoding="utf-8-sig")
files.download("my_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Огляд та очистка даних

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349545 entries, 0 to 349544
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ga_session_id      349545 non-null  Int64  
 1   device             349545 non-null  object 
 2   mobile_model_name  349545 non-null  object 
 3   operating_system   349545 non-null  object 
 4   language           235279 non-null  object 
 5   browser            349545 non-null  object 
 6   continent          349545 non-null  object 
 7   country            349545 non-null  object 
 8   medium             349545 non-null  object 
 9   name               349545 non-null  object 
 10  channel            349545 non-null  object 
 11  date               349545 non-null  dbdate 
 12  item_id            33538 non-null   Int64  
 13  name_1             33538 non-null   object 
 14  category           33538 non-null   object 
 15  price              33538 non-null   float64
 16  sh

Бачимо що Id зараз мають числові типи даних.Оскільки в нашому випадку id складаються тільки з цифр, то можемо так і залишити.  
Змінемо формат дати на datetime.  
Перейменуємо деякі колонки , щоб вони краще відображали суть і не дублювались , як name та name_1

In [11]:
df['date'] = pd.to_datetime(df['date'])

df.rename(columns = {'name': 'traffic_info', 'name_1': 'product_name', 'category': 'product_category'}, inplace = True)

df.head()

,ga_session_id,device,mobile_model_name,operating_system,language,browser,continent,country,medium,traffic_info,...,date,item_id,product_name,product_category,price,short_description,account_id,is_verified,is_unsubscribed,sub-region
0,7268004754,desktop,Chrome,Web,en-gb,Chrome,(not set),(not set),(none),(direct),...,2021-01-19,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>,NaN
1,8321293308,desktop,Chrome,Web,None,Chrome,(not set),(not set),(none),(direct),...,2021-01-06,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>,NaN
2,4392362373,desktop,Chrome,Web,None,Chrome,(not set),(not set),organic,(organic),...,2020-11-05,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>,NaN
3,140108698,desktop,Safari,Web,en-us,Chrome,(not set),(not set),(none),(direct),...,2020-12-09,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>,NaN
4,8351127264,mobile,iPhone,Web,en-us,Safari,(not set),(not set),organic,(organic),...,2020-11-10,<NA>,None,None,NaN,None,<NA>,<NA>,<NA>,NaN


In [12]:
print(f' Kількість рядків та колонок: {df.shape}')
print(f' Числові колонки: {df.select_dtypes(include = np.number).columns}')
print(f' Категоріальні колонки: {df.select_dtypes(exclude = np.number).columns}')

 Kількість рядків та колонок: (349545, 21)
 Числові колонки: Index(['ga_session_id', 'item_id', 'price', 'account_id', 'is_verified',
       'is_unsubscribed'],
      dtype='object')
 Категоріальні колонки: Index(['device', 'mobile_model_name', 'operating_system', 'language',
       'browser', 'continent', 'country', 'medium', 'traffic_info', 'channel',
       'date', 'product_name', 'product_category', 'short_description',
       'sub-region'],
      dtype='object')


In [13]:
print(f' Кількість унікальних сесій: {df['ga_session_id'].nunique()}')

 Кількість унікальних сесій: 349545


In [14]:
print(f' Період за який маємо дані:   {df['date'].min()} - {df['date'].max()}')

 Період за який маємо дані:   2020-11-01 00:00:00 - 2021-01-31 00:00:00


Перевірка текстових значень:

In [15]:
print(f'Девайси: {df["device"].unique()}\n')
print(f'Mobile_model_name: {df["mobile_model_name"].unique()}\n')
print(f'operating_system: {df["operating_system"].unique()}\n')
print(f'language: {df["language"].unique()}\n')
print(f'browser: {df["browser"].unique()}\n')
print(f'continent: {df["continent"].unique()}\n')
print(f'country: {sorted(df["country"].unique())}\n')
print(f'traffic_info: {df["traffic_info"].unique()}\n')
print(f'channel: {df["channel"].unique()}\n')
print(f'product_category: {df["product_category"].unique()}\n')

Девайси: ['desktop' 'mobile' 'tablet']

Mobile_model_name: ['Chrome' 'Safari' 'iPhone' '<Other>' 'Edge' 'ChromeBook' 'Firefox'
 'Pixel 3' 'iPad' 'Pixel 4 XL']

operating_system: ['Web' 'iOS' '<Other>' 'Android' 'Windows' 'Macintosh']

language: ['en-gb' None 'en-us' 'en-ca' 'en' 'zh' 'de' 'es-es' 'fr' 'ko']

browser: ['Chrome' 'Safari' '<Other>' 'Edge' 'Android Webview' 'Firefox']

continent: ['(not set)' 'Asia' 'Africa' 'Europe' 'Oceania' 'Americas']

country: ['(not set)', 'Albania', 'Algeria', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia & Herzegovina', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 

In [16]:
df.select_dtypes(include = 'object').apply(lambda x: x.nunique()) - df.select_dtypes(include = 'object').apply(lambda x: x.str.lower().str.strip().nunique()) # перевіряємо на проблеми з регістром та зайвими пробілами


,0
device,0
mobile_model_name,0
operating_system,0
language,0
browser,0
continent,0
country,0
medium,0
traffic_info,0
channel,0


Бачимо , що немає явних проблем з регістром , пробілами , зайвими сивмолами та іншими критеріями , які можуть спричиняти не зразу помітні дублікати(нариклад: Asia, asia)

Пропущені значення:

In [17]:
mis_values = pd.DataFrame()
mis_values['count'] = df.isnull().sum()
mis_values['percent from total'] = df.isnull().sum() / len(df) * 100.0
mis_values

,count,percent from total
ga_session_id,0,0.000000
device,0,0.000000
mobile_model_name,0,0.000000
operating_system,0,0.000000
language,114266,32.689925
browser,0,0.000000
continent,0,0.000000
country,0,0.000000
medium,0,0.000000
traffic_info,0,0.000000


Найбільше пропущених значень в колонках, які відображають хар-ки товару та регістрацію юзера. Це пов'язано з тим, що тільки в 10 % сесій робиться замовлення і тільки 8% сесій припадає на зареєстрованих користувачів  
Єдине поле , яке має пропущені значення через їх завантаження і якість - це поле language. Тут треба розбиратись з причинами незаповнення цих даних

Дублікати:

In [18]:
df[df.duplicated() == True]

,ga_session_id,device,mobile_model_name,operating_system,language,browser,continent,country,medium,traffic_info,...,date,item_id,product_name,product_category,price,short_description,account_id,is_verified,is_unsubscribed,sub-region


Дублікатів в датасеті немає

Створюємо копію датасета з якою будемо працювати

In [19]:
copy_df = df.copy()

#Аналіз:

Покриття ринків:

In [20]:
print(f'Regions: {copy_df['continent'].nunique()}')
print(f'Sub-regions: {copy_df['sub-region'].nunique()}')
print(f'Countries: {copy_df['country'].nunique()}')

Regions: 6
Sub-regions: 14
Countries: 108


Наша компанія представлена на 6 континентах, в 14 суб-регіонах та в 108-и країнах.  
Можно стверджувати, що компанія представлена по всьому світі


##Аналіз в розрізі регіонів:

Дохід по регіонам:

In [21]:
parametr = ['continent', 'sub-region','country']
parametr_sel = widgets.Dropdown(options = parametr, value = 'continent', description = 'By:')
tops =  ['all'] + list(range(1, 11))
top_sel = widgets.Dropdown(options = tops, value = 'all', description = 'Top:')

##Загальні метрики:

In [22]:

def interactive_vis (par,top):
        grouped_data = copy_df.groupby(par, as_index = False).agg(revenue = ('price', 'sum'),
                                                                    aov = ('price', 'mean'),
                                                                    total_sessions = ('ga_session_id', 'count'),
                                                                    sessions_with_orders = ('product_name', 'count'),
                                                                    register_users = ('account_id', 'count'),
                                                                    sub_count = ('account_id', lambda x: ((copy_df.loc[x.index, 'is_unsubscribed'] == 0) &
                                                                                                          (copy_df.loc[x.index, 'is_verified'] == 0)).sum()),
                                                                    ver_count = ('account_id', lambda x: ((copy_df.loc[x.index, 'is_verified'] == 1) &
                                                                                                           (copy_df.loc[x.index, 'is_unsubscribed'] == 1)).sum()),
                                                                    sub_ver_count = ('account_id', lambda x: ((copy_df.loc[x.index, 'is_verified'] == 1) &
                                                                                                              (copy_df.loc[x.index, 'is_unsubscribed'] == 0)).sum()),
                                                                    other_reg_cnt = ('account_id', lambda x: ((copy_df.loc[x.index, 'is_verified'] == 0) &
                                                                                                              (copy_df.loc[x.index, 'is_unsubscribed'] == 1)).sum())

  )
        grouped_data['% from total'] = grouped_data['revenue'] / copy_df['price'].sum() * 100.0
        grouped_data['conversation_rate'] = grouped_data['sessions_with_orders'] / grouped_data['total_sessions']
        grouped_data['coef_sub'] = grouped_data['sub_count'] / grouped_data['register_users']
        grouped_data['coef_ver'] = grouped_data['ver_count'] / grouped_data['register_users']
        grouped_data['sessions_without_orders'] = grouped_data['total_sessions'] - grouped_data['sessions_with_orders']
        if top != 'all':

          sorted_revenue = grouped_data.sort_values( by ='revenue', ascending = False).head(top)
          sorted_per_from_total = grouped_data.sort_values (by ='% from total', ascending = False).head(top)
          sorted_aov = grouped_data.sort_values(by = 'aov', ascending = False).head(top)
          sorted_sessions = grouped_data.sort_values(by = 'total_sessions', ascending = True).head(top)
          sorted_reg = grouped_data.sort_values(by = 'register_users', ascending = True).head(top)
        else:
          sorted_revenue = grouped_data.sort_values( by ='revenue', ascending = False)
          sorted_per_from_total = grouped_data.sort_values (by ='% from total', ascending = False)
          sorted_aov = grouped_data.sort_values(by = 'aov', ascending = False)
          sorted_sessions = grouped_data.sort_values(by = 'total_sessions', ascending = True)
          sorted_reg = grouped_data.sort_values(by = 'register_users', ascending = True)

        fig1 = go.Treemap(
                         labels=sorted_revenue[par],
                         values=sorted_revenue['revenue'],
                         parents=['']*len(sorted_revenue),
                         marker=dict(
                           colors=sorted_revenue['revenue'],
                           colorscale='blugrn',
                           colorbar=dict(title='Revenue'),
                           showscale = False
                                     ),
                        hovertemplate='<b>%{label}</b><br>Revenue: %{value}<extra></extra>',
                        text=[
                               f"<span style='color:white'><b>{label}</b></span><br>"
                               f"<span style='font-size:10px; color:white'>${val/1e6:.2f}M</span>"
                               for label, val in zip(sorted_revenue[par], sorted_revenue['revenue'])
                         ],
                        textinfo = 'text'
    )

        fig2 = go.Treemap(
                          labels=sorted_per_from_total[par],
                          values=sorted_per_from_total['% from total'],
                          parents=['']*len(sorted_per_from_total),
                          marker=dict(
                                      colors=sorted_per_from_total['% from total'],
                                      colorscale='blugrn',
                                      colorbar=dict(title='% of Total'),
                                      showscale = False
                                      ),
                          text=[
                               f"<span style='color:white'><b>{label}</b></span><br>"
                               f"<span style='font-size:10px; color:white'>{val:.2f}%</span>"
                               for label, val in zip(sorted_per_from_total[par], sorted_per_from_total['% from total'])
                          ],
                          textinfo = 'text',
                          hovertemplate='<b>%{label}</b><br>% of Total: %{value:.2f}%<extra></extra>',

    )
        fig3 = go.Bar(x = sorted_aov[par],
                      y = sorted_aov['aov'],
                      marker = dict(color = sorted_aov['aov'],
                                    colorscale = 'blugrn'),
                      text = (sorted_aov["aov"].round(0)),
                      textposition = 'outside',
                      textfont = dict(color = 'black',
                                      size = 9),
                      cliponaxis=True
        )

        fig4_orders = go.Bar( x = sorted_sessions['sessions_with_orders'], y = sorted_sessions[par], orientation = 'h', showlegend=True, legendgroup='orders',
                             marker_color = 'darkcyan',
                             text = (sorted_sessions['sessions_with_orders'] / sorted_sessions['total_sessions'] * 100).round(2).astype(str) + '%',
                             textposition='inside',
                             textfont =dict(color = 'white',
                                            size = 7),
                             insidetextanchor='middle',
                             textangle=0,
                             hovertext = (sorted_sessions['sessions_with_orders'] / 1000).astype(str) + 'K',
                             hoverinfo = 'text')
        fig4_sessions = go.Bar( x = sorted_sessions['sessions_without_orders'], y = sorted_sessions[par], orientation = 'h', showlegend=True, legendgroup='orders',
                               marker_color = 'seagreen',
                               text = (sorted_sessions['sessions_without_orders'] / sorted_sessions['total_sessions'] * 100).round(2).astype(str) + '%',
                                textposition = 'inside',
                                textfont = dict(color = 'white',
                                                size = 7),
                                insidetextanchor = 'middle',
                                textangle = 0,
                                hovertext = (sorted_sessions['sessions_without_orders'] / 1000).astype(str) + 'K',
                                hoverinfo = 'text')
        fig4_total_sessions = go.Scatter( x = sorted_sessions['total_sessions'] *1.05 + 4000,
                                         y = sorted_sessions[par],
                                         mode='text',
                                         text=(sorted_sessions['total_sessions'] / 1000).round(2).astype(str) + 'K',
                                         textfont=dict(color = 'black',
                                                       size=10),
                                         showlegend=False,
                                         hoverinfo='skip'
                                          )
        fig5_subs = go.Bar( x = sorted_reg['sub_count'], y = sorted_reg[par], orientation = 'h', showlegend=True, legendgroup='users',
                           marker_color = 'darkcyan',
                           text = (sorted_reg['sub_count'] / sorted_reg['register_users'] * 100).round(2).astype(str) + '%',
                           textposition='inside',
                           textfont =dict(color = 'white',
                                          size = 7),
                           insidetextanchor='middle',
                           textangle=0,
                           hovertext = (sorted_reg['sub_count'] / 1000).astype(str) + 'K',
                           hoverinfo = 'text')
        fig5_ver = go.Bar ( x = sorted_reg['ver_count'],  y = sorted_reg[par], orientation = 'h', showlegend=True, legendgroup='users',
                           marker_color = 'mediumaquamarine',
                           text = (sorted_reg['ver_count'] / sorted_reg['register_users'] * 100).round(2).astype(str) + '%',
                           textposition='inside',
                           textfont =dict(color = 'white',
                                          size = 7),
                           insidetextanchor='middle',
                           textangle=0,
                           hovertext = (sorted_reg['ver_count'] / 1000).astype(str) + 'K',
                           hoverinfo = 'text')
        fig5_sub_ver = go.Bar( x = sorted_reg['sub_ver_count'], y = sorted_reg[par], orientation = 'h', showlegend=True, legendgroup='users',
                              marker_color = 'seagreen',
                              text = (sorted_reg['sub_ver_count'] / sorted_reg['register_users'] * 100).round(2).astype(str) + '%',
                              textposition='inside',
                              textfont =dict(color = 'white',
                                             size = 7),
                              insidetextanchor='middle',
                              textangle=0,
                              hovertext = (sorted_reg['sub_ver_count'] / 1000).astype(str) + 'K',
                              hoverinfo = 'text')
        fig5_other_reg = go.Bar( x = sorted_reg['other_reg_cnt'], y = sorted_reg[par], orientation = 'h', showlegend=True, legendgroup='users',
                                marker_color = 'rosybrown',
                                text = (sorted_reg['other_reg_cnt'] / sorted_reg['register_users'] * 100).round(2).astype(str) + '%',
                                textposition='inside',
                                textfont =dict(color = 'white',
                                               size = 7),
                                insidetextanchor='middle',
                                textangle=0,
                                hovertext = (sorted_reg['other_reg_cnt'] / 1000).astype(str) + 'K',
                                hoverinfo = 'text')
        fig5_total_reg = go.Scatter( x=sorted_reg['sub_count'] + sorted_reg['ver_count'] + sorted_reg['sub_ver_count'] + sorted_reg['other_reg_cnt'] * 1.03 + 500,
                                          y=sorted_reg[par],
                                          mode='text',
                                          text=(sorted_reg['register_users'] / 1000).round(2).astype(str) + 'K',
                                          textfont=dict(size=10, color='black'),
                                          showlegend=False,
                                          hoverinfo='skip'
                                   )


        fig = make_subplots(rows=3, cols=2,
                            specs = [[{'type': 'domain'}, {'type': 'domain'}], [{"colspan": 2}, None], [{}, {}]],
                            subplot_titles = (
            f'Revenue by {par}',
            f'% Revenue from Total by {par}',
            f'AOV by {par}',
            f'Sessions & orders by {par}',
            f'Register users(verified & subscribed) by {par}')
        )

        fig.add_trace(fig1, row=1, col=1)
        fig.add_trace(fig2, row=1, col=2)
        fig.add_trace(fig3, row=2, col=1)
        fig.add_trace(fig4_orders, row=3, col=1)
        fig.add_trace(fig4_sessions, row=3, col=1)
        fig.add_trace(fig4_total_sessions, row = 3 , col = 1)

        fig.add_trace(fig5_subs, row=3, col=2)
        fig.add_trace(fig5_ver, row=3, col=2)
        fig.add_trace(fig5_sub_ver, row=3, col=2)
        fig.add_trace(fig5_other_reg, row = 3 , col = 2)
        fig.add_trace(fig5_total_reg, row = 3 , col = 2)

        fig.update_layout(
          title=f'Analysis by {par}',
          height=1000,
          width=1500,
          showlegend= False,
          margin=dict(t=50, b=50, l=25, r=25),
          barmode = 'stack',
          plot_bgcolor = 'aliceblue',
          paper_bgcolor ='lightcyan',
          legend=dict(
            x=0.0,
            y=0.1,
            traceorder="normal",
            font=dict(size=12),
            bgcolor="rgba(255, 255, 255, 0.5)"
        )

    )
        fig.update_xaxes(title_text=par, row=2, col=1)
        fig.update_yaxes(title_text='AOV', row=2, col=1)

        fig.add_annotation(
                           x=0.999, y=0.23,  # Розміщуємо легенду поруч із графіком
                           text=(
                                  '<span style="color:#008080;font-size:18px;">■</span> **verified**<br>'
                                  '<span style="color:#3CB371;font-size:18px;">■</span> **subscribed**<br>'
                                  '<span style="color:#B7E1FC;font-size:18px;">■</span> **other reg**'
                                ),
                           showarrow=False,
                           xref="paper", yref="paper",
                           align="left",
                           bgcolor="rgba(255, 255, 255, 0.8)",
                           font=dict(size=10)
                          )
        fig.add_annotation(
                           x=0.4, y=0.23,  # Розміщуємо легенду поруч із графіком
                           text=(
                                  '<span style="color:#3CB371;font-size:18px;">■</span> **orders**<br>'
                                  '<span style="color:#B7E1FC;font-size:18px;">■</span> **sessions without orders**'
                                ),
                           showarrow=False,
                           xref="paper", yref="paper",
                           align="left",
                           bgcolor="rgba(255, 255, 255, 0.8)",
                           font=dict(size=10)
                          )


        fig.show()

widgets.interact(interactive_vis, par=parametr_sel, top = top_sel)



interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis(par, top)>

**По регіонам:**  

Більше половину доходу приносить континент Америка - 55%.Європа та Азія в сумі приносять близько 42% доходу. Відповідно ці 3 континента формують  основний дохід компанії.   
Така сама ситуація по кількості сесій, замовлень та зарєстрованим користувачам, ці 3 континента формуть майже всю частку показників компанії.  
Також співвідношення кількості сесій до кількості замовлень майже однаково, що свідчить про однаково ефективні стратегії залучення клієнтів.  
Якщо дивитись на середній чек бачимо що він знаходиться в достатньо вузькому діапазоні 950-1000, що свідчить про наявність купівельної спроможності на всіх континентах  
Бачимо , метрики сесій та реєстрацій мають таку саму послідовність регіонів, як і дохід, що може свідчити про взаємозв'язок цих метрик і потребує перевірки.Натомість середній чек може не мати вплива на розмір доходу.  

**По суб-регіонам(топ-10):**  

Поглиблюючись бачимо , що прибуток 55% прибутку формує не просто континент Америка а саме Північна Америка -51 % від доходу.Також цей суб-регіон займає лідерство і по кількості сесій і по кількості зареєстрованих користувачів, формуючи більшу частку цих показників.  
Всі інші суб-регіони мають меньше 10% від загального прибутку кожен.  
Конверсія сесії-замовлення має близькі значення для топ-10 суб-регіонів  

**По країнам(топ-10):**  
Поглиблюючись ще на один рівень, вже бачимо картину, що 42% доходу від 51% доходу суб-регіону формує саме США, тоді як Канада формує  тільки 7.6%.Так само США домінують і по кількості сесій і по кількості зареєстрованих користувачів.  
По іншим регіонам: найприбутковіша країна Азії -Індія 8.8% , Європи - Великобританія - 3%. Країн Африки немає в топ-10.  
На рівні країн теж зберігається майже ідентична послідовність по доходу, по кількості сесій, та по кількості зареєстрованих юзерів.І зберігається візуальна відсутність взаємозв'язку з середнім чеком.


 **Оскільки всього 1 країна(США) формує 42% доходу компанії проведемо ризик- аналіз такого розподілу.  
Також на основі візуальної інформації можно сформувати декілька гіпотез:  
Гіпотеза 1: зі збільшенням кількості сесій  в географічній зоні - зростає дохід.  
Гіпотеза 2: зі збільшенням зареєстрованих користувачів в географічній зоні - зростає дохід.  
Гіпотеза 3: розмір середнього чеку не впливає на дохід в географічній зоні.  
Ці гіпотези ми перевіремо після закінчення аналізу в розділі "Пошук взаємозв'язків"**

##Аналіз популярності девайсів:

In [23]:
def interactive_vis2(par):
  clear_output(wait=True)
  grouped_data = copy_df.groupby([par, 'device'], as_index = False).agg(total_revenue = ('price', 'sum'),
                                          aov = ('price', 'mean'))
  sorted_revenue = grouped_data.groupby(par, as_index = False)['total_revenue'].sum().sort_values(by = 'total_revenue', ascending = False).head(10)
  filter_data = grouped_data[grouped_data[par].isin(sorted_revenue[par])].sort_values(by = 'total_revenue', ascending = False)
  fig1 = px.bar(
      filter_data,
      x = par,
      y = 'total_revenue',
      color = 'device',
      color_discrete_map = {'desktop': 'steelblue',
                            'mobile': 'mediumaquamarine',
                            'tablet': 'wheat'},
      barmode = 'group',
      title = 'Revenue by devices'
  )
  fig2 = px.bar(
      filter_data,
      x = par,
      y = 'aov',
      color = 'device',
      color_discrete_map = {'desktop': 'steelblue',
                            'mobile': 'mediumaquamarine',
                            'tablet': 'wheat'},
      barmode = 'group',
      title = 'AOV by devices'
  )

  fig =make_subplots(rows = 1, cols = 2,
                    subplot_titles = ('Revenue by devices',
                                     'AOV by devices'
                    )
                   )
  for trace in fig1.data:
        trace.showlegend = True
        fig.add_trace(trace, row=1, col=1)

  for trace in fig2.data:
        trace.showlegend = False
        fig.add_trace(trace, row=1, col=2)

  fig.update_layout(title = 'Analysis by Devices',
                    barmode='group',
                    height = 500,
                    width = 1200,
                    showlegend = True,
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan')

  fig.show()

widgets.interact(interactive_vis2, par = parametr_sel)

interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis2(par)>

Найбільше доходу в усіх розрізах нам приносять користувачі, які здійснюють покупку через desktop. Також другий по прибутку mobile теж вносить значну частину доходу до загального  
Якщо ми дивимся на середній чек ситуація інакша.В більшості суб-регіонів розкид середьного чека невеликий, але для Південної та Східної Азії та для Північної Європи явним лідером є середній чек користувачів, які здійснюють покупку через tablet.  
Треба дослідити портрет цих користувачів. Це можуть бути просто заможні люди, або люди певної професії, які купують дорогі товари по робочим потребам , тощо. Після дослідження в разі виявлення паттернів робити акцент в цих суб-регіонах саме на цій групі , оскільки це потенціал для розвитку

##Аналіз популярності браузерів:

In [24]:
def interactive_vis3(par):
  clear_output(wait=True)
  grouped_data = copy_df.groupby([par, 'browser'], as_index = False).agg(total_revenue = ('price', 'sum'),
                                          aov = ('price', 'mean'))
  grouped_data = grouped_data[(grouped_data['browser'] != '<Other>') & (grouped_data[par] != '(not set)')]
  sorted_revenue = grouped_data.groupby(par, as_index = False)['total_revenue'].sum().sort_values(by = 'total_revenue', ascending = False).head(10)
  filter_data = grouped_data[grouped_data[par].isin(sorted_revenue[par])].sort_values(by = 'total_revenue', ascending = False)
  fig1 = px.bar(
      filter_data,
      x = par,
      y = 'total_revenue',
      color = 'browser',
      color_discrete_map = {'Chrome': 'steelblue',
                            'Safari': 'mediumaquamarine',
                            'Edge': 'wheat',
                            'Firefox': 'darksalmon',
                            'Android Webview': 'rosybrown'
                            },
      barmode = 'group',
      title = 'Revenue by browsers'
  )
  fig2 = px.bar(
      filter_data,
      x = par,
      y = 'aov',
      color = 'browser',
      color_discrete_map = {'Chrome': 'steelblue',
                            'Safari': 'mediumaquamarine',
                            'Edge': 'wheat',
                            'Firefox': 'darksalmon',
                            'Android Webview': 'rosybrown'
                            },
      barmode = 'group',
      title = 'AOV by browsers'
  )

  fig =make_subplots(rows = 1, cols = 2,
                    subplot_titles = ('Revenue by browsers',
                                     'AOV by browsers'
                    )
                   )
  for trace in fig1.data:
        trace.showlegend = True
        fig.add_trace(trace, row=1, col=1)

  for trace in fig2.data:
        trace.showlegend = False
        fig.add_trace(trace, row=1, col=2)

  fig.update_layout(title = 'Analysis by Browsers',
                    barmode='group',
                    height = 500,
                    width = 1200,
                    showlegend = True,
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan')

  fig.show()

widgets.interact(interactive_vis3, par = parametr_sel)

interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis3(par)>

Найприбутковішим  в усіх розрізах браузером у є Chrome з сильним відрівом. Також помітну частку займає Safari.  
Оскільки середній чек показує зовсім іншу картину, де лідерами стають Firefox та Android Webview для суб-регіонів та країн Океанії та Афріки, а також по наприбутковішим регіонам всі браузери мають невеликий розкид середнього чеку можно зробити висновок, що прибутковість визначається більшої кількістю відвідувачів через Chrome  а не купівельною спроможність.  
Тому можно дослідити причини настільки більшого середнього чеку для непопулярних браузерів в Океанії та  в Африці, дослідити портрет цих користувачів і налаштувати таргетну рекламу  конкретно під них, а не на загальну  аудиторію

##Аналіз популярності операційних систем:

In [25]:
def interactive_vis4(par):
  clear_output(wait=True)
  grouped_data = copy_df.groupby([par, 'operating_system'], as_index = False).agg(total_revenue = ('price', 'sum'),
                                          aov = ('price', 'mean'))
  grouped_data = grouped_data[(grouped_data['operating_system'] != '<Other>') & (grouped_data[par] != '(not set)')]
  sorted_revenue = grouped_data.groupby(par, as_index = False)['total_revenue'].sum().sort_values(by = 'total_revenue', ascending = False).head(10)
  filter_data = grouped_data[grouped_data[par].isin(sorted_revenue[par])].sort_values(by = 'total_revenue', ascending = False)
  fig1 = px.bar(
      filter_data,
      x = par,
      y = 'total_revenue',
      color = 'operating_system',
      color_discrete_map = {'Web': 'steelblue',
                            'Windows': 'mediumaquamarine',
                            'iOS': 'wheat',
                            'Android': 'darksalmon',
                            'Macintosh': 'rosybrown'
                            },
      barmode = 'group',
      title = 'Revenue by OS'
  )
  fig2 = px.bar(
      filter_data,
      x = par,
      y = 'aov',
      color = 'operating_system',
      color_discrete_map = {'Web': 'steelblue',
                            'Windows': 'mediumaquamarine',
                            'iOS': 'wheat',
                            'Android': 'darksalmon',
                            'Macintosh': 'rosybrown'
                            },
      barmode = 'group',
      title = 'AOV by OS'
  )

  fig =make_subplots(rows = 1, cols = 2,
                    subplot_titles = ('Revenue by OS',
                                     'AOV by OS'
                    )
                   )
  for trace in fig1.data:
        trace.showlegend = True
        fig.add_trace(trace, row=1, col=1)

  for trace in fig2.data:
        trace.showlegend = False
        fig.add_trace(trace, row=1, col=2)

  fig.update_layout(title = 'Analysis by OS',
                    barmode='group',
                    height = 500,
                    width = 1200,
                    showlegend = True,
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan')

  fig.show()

widgets.interact(interactive_vis4, par = parametr_sel)

interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis4(par)>

Тут найприбутковішою категорією є Web в усіх розрізах з сильним відривом(в усіх інших операційних систем в сумі прибуток меньше ніж у Web). Це означає, що більшість наших користувачів роблять покупку не через певні додатки, а через вебінтерфейс.  
Середній чек для трьох найприбутковіших регіонів немає відчутніх змін між операційними системами.Тому додатково вкладання грошей в розвиток і рекламу додатків не призведе до збільшення прибутку і є неефективним.  
Водночас для Oкеанії та АзіЇ(в особливості для Китая та Сингапура) середній чек найбільший для непопулярної системи Macintosh. Тут теж треба дослідити портрет цих юзерів і робити акцент в рекламі саме на них.

##Аналіз ефективності каналів трафіка:

In [26]:
def interactive_vis5(par):
  grouped_data = copy_df.groupby([par, 'channel'], as_index = False).agg(total_revenue = ('price', 'sum'),
                                          aov = ('price','mean'),
                                          conv_rate =( 'product_name', lambda x: x.notna().mean())
                                          )
  sorted_revenue = grouped_data.groupby(par, as_index = False)['total_revenue'].sum().sort_values(by = 'total_revenue', ascending = False).head(10)
  filter_data = grouped_data[grouped_data[par].isin(sorted_revenue[par])].sort_values(by ='total_revenue', ascending = False)
  pivot_data_aov = filter_data.pivot_table(index = par,
                                        columns = 'channel',
                                        values = 'aov')
  pivot_data_conv = filter_data.pivot_table(index = par,
                                        columns = 'channel',
                                        values = 'conv_rate')

  fig1 = px.bar(
      filter_data,
      x = par,
      y ='total_revenue',
      color = 'channel',
      color_discrete_map = {'Organic Search': 'steelblue',
                            'Paid Search': 'mediumaquamarine',
                            'Direct': 'wheat',
                            'Social Search': 'darksalmon',
                            'Undefined': 'rosybrown'
                            },
      barmode = 'group'

  )
  fig2 = go.Heatmap(z = pivot_data_aov.values,
                    x = pivot_data_aov.columns,
                    y = pivot_data_aov.index,
                    text = pivot_data_aov.values.round(0),
                    texttemplate = '%{text}',
                    textfont = dict(size = 8),
                    colorscale = 'GnBU',
                    showscale = False)
  fig3 = go.Heatmap( z = pivot_data_conv.values,
                     x = pivot_data_conv.columns,
                     y = pivot_data_conv.index,
                     text = pivot_data_conv.values.round(4),
                     texttemplate = '%{text}',
                     textfont = dict(size = 8),
                     colorscale = 'GnBU',
                     showscale = False)
  fig = make_subplots(rows = 2, cols = 2,
                      specs = [[{"colspan": 2}, None], [{}, {}]],
                      subplot_titles = ('Revenue by Channel',
                                        'AOV by Channel',
                                        'Conversion by Channel'
                                       )
                     )
  for trace in fig1.data:
      trace.showlegend = True
      fig.add_trace(trace, row=1, col=1)
  fig.add_trace(fig2, row=2, col=1)
  fig.add_trace(fig3, row=2, col=2)

  fig.update_yaxes(
      tickangle = 45,
      tickfont = dict(size = 10),
      row =2,
      col =1
  )
  fig.update_yaxes(
      tickangle = 45,
      tickfont = dict(size = 10),
      row = 2,
      col = 2
  )

  fig.update_layout(title =f'Analysis by Channels({par})',
                    height = 1000,
                    width = 1300,
                    barmode = 'group',
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan')
  fig.show()

widgets.interact(interactive_vis5, par = parametr_sel)

interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis5(par)>

**Organic Search**

Найбільший генератор доходу в усіх розрізах, особливо у  розрізі регіону Північної Америки (~6M).

У поєднанні з середнім AOV і стабільною конверсією це базовий канал для всіх ринків.

Варто підтримувати як основу, інвестуючи у SEO та контент.

**Paid Search**

Значний внесок у дохід у загальному та особливо у Північній Америці (~4.2M) та Європі (Західна, Східна, Північна – по 0.5–0.7M кожна).

В цих регіонах він ефективний і по конверсії, і по AOV.Варто підтримувати на тому самому рівні.

У Латинській Америці, Азії та Західній Азії дохід мінімальний.Канал там неефективний, краще перерозподілити інвестиції.

**Direct**

Дає великий дохід у Північній Америці (4M) та стабільний у всіх інших регіонах (~0.3–0.7M).

Має середній AOV і стабільну конверсію.Канал підтримки, який працює всюди.

**Social Search**

Хоч і показує найвищі показники AOV та конверсії, але по доходу відстає: у Північній Америці лише ~1.3M, в інших регіонах ще менше.

Це означає, що аудиторія тут поки невелика, але потенціал високий. Масштабування трафіка  можне суттєво збільшити дохід.




##Аналіз прибутковості категорій:

Загальний прибуток по категоріям:

In [27]:
def interactive_pivot(par):
  pivot_data = copy_df.pivot_table(index = par, columns = 'product_category', values = 'price', aggfunc = 'sum', fill_value = 0)
  pivot_data_sort = pivot_data.reindex(pivot_data.sum(axis=1).sort_values(ascending=False).index).round(0)
  styled_data = pivot_data.style.highlight_max(axis = 1, color = 'lightskyblue').format('{:,.0f}$')
  return styled_data
widgets.interact(interactive_pivot, par = parametr_sel)

interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_pivot(par)>

Голубим кольором відзначена найприбутковіша категорія для кожної географічної зони

**Бачимо що в розрізі континентів та суб-регіонів по всім елементам найбільш прибутковою категорією є Sofas & amrchairs.**

 **В розрізі країн ця тенденція для великою кількості країн зберігається,але також в багатьох країнах лідерство займають інші категорії.  
Наприклад в Румунії 32000$ приносить категорія Chairs , що вдвічі більше за будь-яку іншу категорію в цій країні.Також ця категорія є лідером для Турції- 79 800, Південної Кореї- 88 400,Японії- 96 300 та для багатьох інших.**  

**В жодній країні лідерами не є такі категорії:Chests of drawers & drawer units,	Children's furniture,	Nursery furniture,Room dividers.**


Побудуєм графік індекса афінності, щоб зрозуміти, в яких географічних зонах категорії мають найбільшу популярність

Індекс афінності:

In [28]:
def interactive_vis6(par):
  grouped_data = copy_df.groupby([par, 'product_category'], as_index = False)['price'].sum()
  sorted_revenue = copy_df.groupby(par, as_index = False)['price'].sum().sort_values(by = 'price', ascending = False).head(10)
  filter_data = grouped_data[grouped_data[par].isin(sorted_revenue[par])]
  pivot_data = filter_data.pivot_table( index = par,
                                        columns = 'product_category',
                                        values = 'price',
                                        fill_value = 0
                                        )
  pivot_data = pivot_data.round(2)

  category_part = pivot_data.apply(lambda row: row / row.sum(), axis = 1)
  global_category_part = copy_df.groupby('product_category')['price'].sum() / copy_df['price'].sum()
  affinity_index = category_part.div(global_category_part, axis = 1) # афініті індекс для хітмапи


  fig = go.Figure(
                    data = go.Heatmap( z = affinity_index.values,
                                       x = affinity_index.columns,
                                       y = affinity_index.index,
                                       text = affinity_index.values.round(2),
                                       texttemplate = '%{text}',
                                       textfont = dict(size = 8),
                                       colorscale = 'GnBU',
                                       showscale = False)
  )


  fig.update_layout( title = f'Afinnity Index by Category({par})',
                    height = 700,
                    width = 800,
                    barmode = 'group',
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan'
                   )
  fig.show()

widgets.interact(interactive_vis6, par = parametr_sel)


interactive(children=(Dropdown(description='By:', options=('continent', 'sub-region', 'country'), value='conti…

<function __main__.interactive_vis6(par)>

Що таке афініті індекс  - показує, наскільки краще, частіше або щільніше цільова аудиторія контактує з  продуктом, ніж уся базова аудиторія.Тобто це % прибутку категорії від усіх категорій географічної зони поділений на % прибутку цією категорії від загального прибутку.

**Континентальні особливості:**  

Америка,Європа та Азія показують стабільний індекс по всім категоріям.Це гарне поле для тестування нової продукції та масштабування.  
Океанія та Африка потребують максимізування маржі на спеціалізованих найбільш популярних категоріях таких як Bar furniture.  
**Регіональні особливості:**  

Північна Америка - збалансований профіль споживання (індекси 0.88-1.06).
Стабільний, передбачуваний ринок  
Північна Європа - найвищий попит на outdoor furniture (1.77), може бути пов'язано з культурою скандинавських країн життя на природі  
Південна Європа - лідирує в room dividers (1.99).  
Західна Європа - збалансований попит з піком в room dividers (1.84)  
Західна Азія -yнікальна перевага: Chests of drawers (1.17) - найвища серед азійських регіонів.  
Південно-східна Азія - сильний фокус на Bar furniture (1.36) та Room dividers (1.33-1.38).  
Південна Азія - найбільш збалансований азійський ринок.Tables & desks (1.22) - найвищий показник.
Східна Азія - Сильні позиції: Children's furniture (1.25), Chests of drawers (1.13).  
**Найсильніші ринки за категоріями:**  

*Room dividers*:
Німеччина (3.02) та Іспанія (2.65) показують надзвичайно високий попит  
*Outdoor furniture*:
Велика Британія (2.06) та Китай (1.65) лідирують.
Канада (1.48) також показує хороший результат.  
**"(not set)" - найбільша аномалія:**  
Індекс 11.46 для Room dividers по континентам може свідчити або по проблеми з збором та обробкою даних або про величезну можливість для розвитку.Потребує дослідження










#Пошук залежностей:

Підготовка даних:

In [29]:
continent_data = copy_df.groupby('continent').agg(total_revenue = ('price', 'sum'),
                                                    aov = ('price', 'mean'),
                                                    total_sessions = ('ga_session_id', 'count'),
                                                    reg_users = ('account_id', 'count')
                                                    )
subregion_data = copy_df.groupby('sub-region').agg(total_revenue = ('price', 'sum'),
                                                    aov = ('price', 'mean'),
                                                    total_sessions = ('ga_session_id', 'count'),
                                                    reg_users = ('account_id', 'count')
                                                    )
country_data = copy_df.groupby('country').agg(total_revenue = ('price', 'sum'),
                                                    aov = ('price', 'mean'),
                                                    total_sessions = ('ga_session_id', 'count'),
                                                    reg_users = ('account_id', 'count')
                                                    )

##Перевірка нормальності розподілу даних:

Для континентів:

In [30]:
stat, p_value_rev = st.shapiro(continent_data['total_revenue'])
stat, p_value_aov = st.shapiro(continent_data['aov'])
stat, p_value_ses = st.shapiro(continent_data['total_sessions'])
stat, p_value_reg = st.shapiro(continent_data['reg_users'])

print(f'p revenue: {p_value_rev}\np aov: {p_value_aov}\np sessinos: {p_value_ses}\np register users: {p_value_reg}')

p revenue: 0.08085648833572931
p aov: 0.2169278796597206
p sessinos: 0.08008979328482427
p register users: 0.0785520646713255


Всі p-value > 0.05 , що свідчить про нормальний розподіл

Для суб-регіонів:

In [31]:
stat, p_value_rev = st.shapiro(subregion_data['total_revenue'])
stat, p_value_aov = st.shapiro(subregion_data['aov'])
stat, p_value_ses = st.shapiro(subregion_data['total_sessions'])
stat, p_value_reg = st.shapiro(subregion_data['reg_users'])

print(f'p revenue: {p_value_rev}\np aov: {p_value_aov}\np sessinos: {p_value_ses}\np register users: {p_value_reg}')

p revenue: 5.083994173352863e-06
p aov: 0.0033369556786995032
p sessinos: 5.307012256581308e-06
p register users: 5.398913759715056e-06


Всі p-value < 0.05 , що свідчить про ненормальний розподіл

Для країн:

In [32]:
stat, p_value_rev = st.shapiro(country_data['total_revenue'])
stat, p_value_aov = st.shapiro(country_data['aov'])
stat, p_value_ses = st.shapiro(country_data['total_sessions'])
stat, p_value_reg = st.shapiro(country_data['reg_users'])

print(f'p revenue: {p_value_rev}\np aov: {p_value_aov}\np sessinos: {p_value_ses}\np register users: {p_value_reg}')

p revenue: 6.940722856167402e-22
p aov: 0.0003097408596627876
p sessinos: 6.637939518689142e-22
p register users: 6.495252202264297e-22


Всі p-value < 0.05 , що свідчить про ненормальний розподіл


**За результатами перевірки для регіонів будем використовувати кореляцію пірсона, а для суб-регіонів та країн - спірмен**а

##Перевірка залежності доходу від середнього чеку, кількості сесій  та кількості реєстрацій:

Кореляція в розрізі регіонів:

In [33]:
con_corr_aov, p_value_aov = st.pearsonr(continent_data['aov'], continent_data['total_revenue'])
con_corr_ses, p_value_ses = st.pearsonr(continent_data['total_sessions'], continent_data['total_revenue'])
con_corr_reg, p_value_reg = st.pearsonr(continent_data['reg_users'], continent_data['total_revenue'])

print(f'Cor between aov and revenue: {con_corr_aov}\nCor between sessions and revenue: {con_corr_ses}\nCor between registrations and revenue: {con_corr_reg}\n')
print(f'p-value aov: {p_value_aov}\np-value ses: {p_value_ses}\np-value reg: {p_value_reg}')

Cor between aov and revenue: -0.666936528889279
Cor between sessions and revenue: 0.9999977187918115
Cor between registrations and revenue: 0.9999965886949237

p-value aov: 0.1479233357980835
p-value ses: 7.805860263798734e-12
p-value reg: 1.745548363563065e-11


Кореляція в розрізі суб-регіонів:

In [34]:
sub_corr_aov, p_value_aov = st.spearmanr(subregion_data['aov'], subregion_data['total_revenue'])
sub_corr_ses, p_value_ses = st.spearmanr(subregion_data['total_sessions'], subregion_data['total_revenue'])
sub_corr_reg, p_value_reg = st.spearmanr(subregion_data['reg_users'], subregion_data['total_revenue'])

print(f'Cor between aov and revenue: {sub_corr_aov}\nCor between sessions and revenue: {sub_corr_ses}\nCor between registrations and revenue: {sub_corr_reg}\n')
print(f'p-value aov: {p_value_aov}\np-value ses: {p_value_ses}\np-value reg: {p_value_reg}')

Cor between aov and revenue: -0.3758241758241759
Cor between sessions and revenue: 0.9956043956043955
Cor between registrations and revenue: 0.9956043956043955

p-value aov: 0.185410028697123
p-value ses: 1.0315953358848785e-13
p-value reg: 1.0315953358848785e-13


Кореляція в розрізі країн:

In [35]:
coun_corr_aov, p_value_aov = st.spearmanr(country_data['aov'], country_data['total_revenue'])
coun_corr_ses, p_value_ses = st.spearmanr(country_data['total_sessions'], country_data['total_revenue'])
coun_corr_reg, p_value_reg = st.spearmanr(country_data['reg_users'], country_data['total_revenue'])

print(f'Cor between aov and revenue: {coun_corr_aov}\nCor between sessions and revenue: {coun_corr_ses}\nCor between registrations and revenue: {coun_corr_reg}\n')
print(f'p-value aov: {p_value_aov}\np-value ses: {p_value_ses}\np-value reg: {p_value_reg}')


Cor between aov and revenue: 0.358474568197624
Cor between sessions and revenue: 0.9669842330150515
Cor between registrations and revenue: 0.9607744011723133

p-value aov: 0.0001392809954940775
p-value ses: 9.252197802979456e-65
p-value reg: 7.297249328839334e-61


В усіх трьох розрізах видно, що є дуже сильний взаємозв'язок між доходом та кількістю сесій та між доходом та кількістю зареєстрованих користувачів. Обидва ці зв'язки є статистично значущими (p-value < 0.05)  
При тому середній чек не показує ніякого зв'язку з позицією регіонів, суб-регіонів або країн відносно загального доходу.

##Перевірка динамічної залежності продажів певної категорії між усіма країнами певного суб-регіону

Віджети:

In [36]:
subregions = copy_df['sub-region'].unique()
sub_sel = widgets.Dropdown(options = subregions, value = 'Northern America', description = 'Sub-region:')
categories = copy_df['product_category'].unique()
cat_sel = widgets.Dropdown(options = categories, value = 'Chairs', description ='Category:')

In [37]:
def calculate_p_values(df):
    df_columns = df.columns
    p_matrix = pd.DataFrame(index=df_columns, columns=df_columns)

    for col1 in df_columns:
        for col2 in df_columns:
            corr, p_value = st.spearmanr(df[col1], df[col2])
            p_matrix.loc[col1, col2] = p_value

    return p_matrix.astype(float)

def interactive_cor(sub, cat):
  clear_output(wait = True)
  filter_data = copy_df[(copy_df['sub-region'] == sub) & (copy_df['product_category'] == cat) & (copy_df['price'].notna())]
  grouped_data = filter_data.groupby(['country', pd.Grouper(key = 'date', freq = 'ME')], as_index = False)['price'].sum()
  pivot_data = grouped_data.pivot_table(index = 'date', columns = 'country', values = 'price', fill_value = 0)
  correlation_matrix = pivot_data.corr()
  pvalue_matrix = calculate_p_values(pivot_data)

  fig = make_subplots(rows = 1, cols = 2,
                    subplot_titles=(f'Cor matrix for {cat} in {sub}',
                                    f'P-Value for {cat} in {sub}'),
        horizontal_spacing=0.1
                    )

  fig.add_heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        zmin=-1,
        zmax=1,
        colorscale='GnBu',
        showscale=False,
        text=np.round(correlation_matrix.values, 3),
        texttemplate="%{text}",
        textfont={"size": 8},
        row=1, col=1
    )

  fig.add_heatmap(
        z=pvalue_matrix.values,
        x=pvalue_matrix.columns,
        y=pvalue_matrix.index,
        zmin=0,
        zmax=0.1,
        colorscale='Blues_r',
        showscale=False,
        text=np.round(pvalue_matrix.values, 4),
        texttemplate="%{text}",
        textfont={"size": 8},
        row=1, col=2
    )

  fig.update_layout(height = 600,
                    width = 1200,
                    plot_bgcolor = 'aliceblue',
                    paper_bgcolor ='lightcyan'
                    )
  fig.show()

widgets.interact(interactive_cor, sub = sub_sel, cat = cat_sel)

interactive(children=(Dropdown(description='Sub-region:', index=13, options=(nan, 'Eastern Asia', 'Southern As…

<function __main__.interactive_cor(sub, cat)>

Справа на графіку - матриця кореляцій яка показує взаємозв'язок продажів вибраної  категорій по країнам певного суб-регіону.  
Зліва на графіку - матриця p-value, яка дозволяє нам зрозуміти статистичну значущість відповідної кореляції. Якщо значення < 0.05 ми будемо вважати кореляцію такою, що варта уваги й ймовірно не є випадковою.

###Візуальне нотування кореляцій в топ -3 суб-регіонах за доходом :

Америка Канада - Cabinets & cupboards,Tables & desks,Beds,Bookcases & Shelving units, Sofas & Armchairs,Childrens Funiture. Cafe furniture -зворотня   
Southern Asia, Cabinets & cupboards -Індія бангладеш зворотня дуже сильна ,  
Tables & desks -Пакістан Бангдалеш , Шрі ланка Індія зворотня сильна,  
Chairs - Непал Шріланка , Пакістан індія зворотня сильна,  
Beds - Пакістан Індія сильна пряма,  
Bookcases & Shelving units - Шріланка Непал - сильна пряма,  
Outdoor funiture - Пакістан Бангладеш зворотня сильна,  
Sofas & armchairs - Пакістан Бангладеш, Індія Бангладеш зворотня сильна.Пакістан Індія пряма ідеальна.  
Childrens Funiture - Пакістан Індія майже ідеальна пряма.  
Chests of drawers & drawer - Індія Пакістан дуже сильна пряма.  
Bar Funiture - Індія Пакістан дуже сильна пряма.  

Eastern Asia:  
Cabinets & cupboards - Південна Корея Хон Конг зворотня , Південна Корея Японія дуже сильна пряма.  
Tables & desks - Макао,Тайвань, Південна Корея  з Хон Конгом дуже сильна пряма,Монголія Японія дуже сильна зворотня.  
Chairs - Південна Корея Китай дуже сильна зворотня,Японія Хон Конг пряма.  
Beds - Тайвань з Хон Конг та Японією дуже сильна пряма, Японія з Хонконгом дуже сильна пряма.  
Bookcases & Shelving units - Тайвань Китай сильна пряма.Південна Корея з Хонконг Японією та Макао дуже сильна зворотня.Макао Хонконг та Макао Японія дуже сильна пряма. Японія Хон Конгмайже ідеальна пряма.  
Nursery Funiture - Японія з Китаємо дуже сильна зворотня.  
Outdoor funiture - Південна Корея Хон Конг майже ідеальна зворотня, Південна Корея Японія майже ідеальна пряма.Японія Хон Конг майже ідеальна зворотня.  
Sofas & armchairs - Тайвань Хон Конг дуже сильна зворотня. Південна Корея Японія пряма.
Cafe Furniture - Тайвань Південна Корея дуже сильна пряма.  
Childrens Funiture  -  Тайвань та Південна Корея з Хон Конгом дуже сильна пряма. Тайвань Південна Корея майже ідеальна пряма.  
Chests of drawers & drawer - Південна Корея Китай сильна пряма.Тайвань Японія дуже сильна зворотня.  
Bar Funiture -  Тайвань з Китаєм та Японією дуже сильна зворотня.Японія з Китаєм і Хон Конгом сильна пряма.Хон Конг з Китаєм пряма.  
Sideboards, buffets & console tables - Південна Корея Хонконг майже ідеальна пряма.  
Room dividers - Південна Корея Тайвань майже ідеаьна пряма.Хон Конг Китай майже ідеальна пряма.   



### Висновки по кореляціям,які підтверджуться статистичною значущостю на основі записів:

**Північна америка:**  
Попит на Cabinets & Cupboards, Tables & Desks, Beds, Bookcases & Shelving Units, Sofas & Armchairs, Childrens Furniture рухається синхронно між США та Канадою. Тобто якщо якась категорія/товар з цієї категорії показує успіх в одній країні, висока ймовірність повторення цього тренду в іншій.  
Cafe Furniture  між США та Канадою - якщо зростає в одній країні, то інша може показати спад.Це означає різні цикли розвитку в цій сфері.  
  
**Південна Азія:**  
Є низка сильних прямих пар (наприклад Beds: Пакистан–Індія, Childrens Furniture: Пакистан–Індія, Chests of Drawers & Drawer / Bar Furniture: Індія–Пакистан). Це означає: якщо категорія зростає в одній з цих країн, то варто швидко масштабувати її на партнерську.  
Водночас є сильні зворотні зв’язки (наприклад Tables & Desks: Пакистан–Бангладеш, Chairs: Непал–Шрі-Ланка). Тут важливо не дублювати стратегії: зростання в одному ринку часто означає падіння в іншому.Це дає змогу вибудовувати дзеркальні стратегії.  

**Східна Азія:**  
Є ідеальні прямі зв’язки (Beds: Тайвань–Хонконг, Японія–Хонконг; Tables & Desks: Макао–Тайвань, Корея–Хонконг), що дозволяють прогнозувати попит і будувати «ланцюговий ефект».  
Одночасно є дуже сильні зворотні кореляції (Chairs: Корея–Китай, Chests of Drawers: Тайвань–Японія, Bar Furniture: Тайвань–Японія). Це вказує на те, що ці ринки конкурують за тренди – споживачі швидко переключаються.  
Для Outdoor Furniture, Cafe Furniture, Childrens Furniture часто фіксуються майже ідеальні прямі кореляції (наприклад Тайвань–Корея, Корея–Японія). Це означає, що  ці сегменти тут ростуть синхронно й можна масштабувати успішні кампанії одночасно в кількох країнах.  

**Варто враховувати:**   
Зі збільшенням даних по продажам, ми зможемо отримати більше важливих кореляцій за рахунок більшої кількості статистично значущих результатів.





#Загальні Висновки:

Компанія формує основний дохід за рахунок трьох континентів - Америки, Європи та Азії, при цьому Північна Америка є беззаперечним лідером (понад половина загального доходу). США виступають ключовим драйвером бізнесу, формуючи майже половину виручки компанії, що створює як значну точку зростання, так і ризик концентрації на одному ринку. Європейські та азійські країни дають менші частки, але забезпечують диверсифікацію, особливо Індія та Велика Британія.

В усіх розрізах чітко простежується: дохід прямо залежить від кількості сесій і зареєстрованих користувачів, тоді як середній чек не має суттєвого впливу на результат. Це означає, що основна стратегія зростання повинна ґрунтуватися на масштабуванні трафіку та залученні нових користувачів, а не на підвищенні середнього чеку.

Найбільшу частину доходу генерують користувачі з desktop, але mobile теж має значний потенціал. Tablet в окремих субрегіонах (Південна та Східна Азія, Північна Європа) показує високий середній чек, що вимагає додаткового дослідження аудиторії. Аналогічно варто дослідити сегменти користувачів з непопулярними браузерами та системами (наприклад, Firefox, Macintosh), які демонструють високий середній чек у специфічних регіонах.

Канали трафіку мають чітку ієрархію: Organic Search - ключовий драйвер доходу, Paid Search ефективний переважно у Північній Америці та Європі, Direct стабільно працює всюди, а Social Search поки відстає за обсягами, але має високий потенціал масштабування.

За категоріями найбільш прибутковою глобально є Sofas & Armchairs, однак у багатьох країнах домінують інші сегменти (наприклад, Chairs у Туреччині, Японії, Південній Кореї). Це свідчить про важливість локальних адаптацій асортименту. Додатково афінність по регіонах відкриває перспективні ніші: Outdoor Furniture у Північній Європі, Room Dividers у Південній Європі, Bar Furniture у Південно-Східній Азії.

Кореляційний аналіз між країнами в межах субрегіонів показав:

У Північній Америці більшість категорій рухаються синхронно між США та Канадою.

У Південній Азії є як прямі, так і зворотні пари, що дозволяє прогнозувати попит та будувати дзеркальні стратегії.

У Східній Азії ринок найбільш динамічний: тут одночасно існують ідеальні прямі та сильні зворотні залежності, що створює як можливості масштабування, так і потребу в гнучкому управлінні асортиментом.

Загалом компанія має стійку основу в Північній Америці та перспективи росту в Азії й Європі за рахунок правильної локалізації, оптимізації трафіку та використання міжкраїнових кореляцій для прогнозування попиту. Наступним кроком є глибше дослідження специфічних сегментів користувачів і перевірка нових категорій у регіонах із високим індексом афінності.

#Дашборд для більш зручного візуального аналізу всіх результатів:

**Як користуватись:**

Під заголовком по центру є 4 фільтри:  
1. Geographic Zone - визначає рівень агрегації для всіх графіків, тобто ми можемо подивитись інформацію в трьох розрізах на вибір: континенти, суб-регіони, країни  
2. Elements in Geographic Zone - тут показуються всі елементи, які входять до вибраного розріза, тобто для континентів тут буде список континентів і т.д.  
3. Date - фільтр дати, де рухая повзунок можно побачити інформацію тільки за певний період.  
4. TOP -  цей фільтр дозволяє нам залишити певний топ елементів за доходом. Якщо ми хочем повернути всі елементи можно просто натиснути ALL.

**Рекомендовано користуватись фільтрами в тій же послідовності в якій вони розташовані на дашборді.**

**Посилання:**

https://public.tableau.com/views/GeographicalAnalysis_17590589104860/Geographicanalysise-commerce?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link